# GridLAB-D Event handlers

In [1]:
!cat developer/session7_2.glm

#set pythonpath=developer
module session7_2;

class example
{
    randomvar real_power[MW];
    randomvar power_factor;
    complex power[MVA];
    
    on_init "python:session7_2.example_init";
    on_commit "python:session7_2.example_commit";
}

#define YEAR=2022
#define MONTH=07
clock 
{
    timezone "PST+8PDT";
    starttime "${YEAR}-${MONTH}-01 00:00:00 PDT";
    stoptime "${YEAR}-${MONTH}-01 02:00:00 PDT";
}

object example
{
    name "main";
    object example:..5
    {
        name `sub_{id}`;
        real_power "type:lognormal(0,1); refresh:1h";
        power_factor "type:triangle(0.85,1.0);refresh:1h";
        on_sync "python:session7_2.sub_sync";
    };
    on_presync "python:session7_2.main_presync";
    on_postsync "python:session7_2.main_postsync";
}


In [2]:
!cat developer/session7_2.py

# session7_2.py

import sys, math, datetime

real_power = {}
power_factor = {}
power = {}

def on_precommit(t):
    print(f"\nEVENT: on_precommit(t={t})",file=sys.stderr)
    print(f"\n*** advancing clock to {datetime.datetime.fromtimestamp(t)} ***",file=sys.stderr)
    return gridlabd.NEVER

#
# Class event handlers
#

def example_init(obj,t):
    print(f"\nEVENT: {obj}.on_init(t={t})",file=sys.stderr)

    # create the accessors
    power[obj] = gridlabd.property(obj,'power')
    real_power[obj] = gridlabd.property(obj,'real_power')
    power_factor[obj] = gridlabd.property(obj,'power_factor')

    # update the power values
    update_power(obj)

    return False # !!!

def example_commit(obj,t):
    print(f"\nEVENT: {obj}.on_commit(t={t})",file=sys.stderr)
    print(f"       {obj}.power = {power[obj]}",file=sys.stderr)
    return gridlabd.NEVER

#
# Object event handlers
#

def main_presync(obj,t):
    print(f"\nEVENT: {obj}.on_presync(t={t})",file=sys.stderr)

    # reset the main 

In [3]:
import gridlabd
gridlabd.command('developer/session7_2.glm')
gridlabd.start('wait')


EVENT: main.on_init(t=1656658800)
       main.power: S not changed

EVENT: sub_1.on_init(t=1656658800)
       sub_1.power: P=1.273855, PF=0.93 -> S=1.273855+0.496918j

EVENT: sub_2.on_init(t=1656658800)
       sub_2.power: P=0.399902, PF=0.95 -> S=0.399902+0.133944j

EVENT: sub_3.on_init(t=1656658800)
       sub_3.power: P=0.748694, PF=0.90 -> S=0.748694+0.373060j

EVENT: sub_4.on_init(t=1656658800)
       sub_4.power: P=2.783489, PF=0.94 -> S=2.783489+1.048929j

EVENT: sub_5.on_init(t=1656658800)
       sub_5.power: P=2.069196, PF=0.96 -> S=2.069196+0.632533j

EVENT: on_precommit(t=1656658800)

*** advancing clock to 2022-07-01 00:00:00 ***

EVENT: main.on_presync(t=1656658800)
       main.power = +0+0j MVA

EVENT: sub_1.on_sync(t=1656658800)
       sub_1.power: P=1.573176, PF=0.89 -> S=1.573176+0.820106j
       main.power = +1.57318+0.820106j MVA

EVENT: sub_2.on_sync(t=1656658800)
       sub_2.power: P=2.879780, PF=0.95 -> S=2.879780+0.917322j
       main.power = +4.45296+1.73743j 

0